# Importing libraries, loading and transforming data

In [ ]:
!pip install -q evaluate transformers==4.28.1
!pip install -U -q datasets
!pip install -q torchaudio==0.12
!add-apt-repository -y ppa:savoury1/ffmpeg4
!apt-get -qq install -y ffmpeg
!pip install -q mlflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.12.0 which is incompatible.
Repository: 'deb https://ppa.launchpadcontent.net/savoury1/ffmpeg4/ubuntu/ jammy main'
Description:
FFmpeg 4.4.4 builds (& associated multimedia packages) for Xenial & newer.

*** Anyone interested in full builds of FFmpeg 4.4.x including all "bells and whistles" needs to have donated, after which access to the new private PPA can be requested. See my Launchpad page linked below for details. ***

*** Please always see https://launchpad.net/~savoury1 for general updates about this Launchpad site before contacting me or reporting any bugs! ***

*** Big thanks to all those who have donated to support this project, you are very directly helping to keep it alive! To all who have not donated: please do so if you can afford it, this p

In [ ]:
pip show tensorflow

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, jax, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: explainable-ai-sdk, tensorflow-cloud, tensorflow-decision-forests, tensorflow-serving-api, tensorflow-text, tensorflowjs, witwidget
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#imports
import pandas as pd
import gc
import re
import numpy as np
import torch
from imblearn.over_sampling import RandomOverSampler
import datasets
import transformers
print(transformers.__version__)

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


4.28.1


In [ ]:
RATE_HZ = 16000 # resampling rate in Hz
MAX_LENGTH = 48000 # maximum audio interval length to consider (= RATE_HZ * SECONDS)
labels = ['Adhunik','Folk','Indie','Islamic','Hiphop','Rock','Metal','Pop']
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

print(id2label, '\n\n', label2id)

{0: 'Adhunik', 1: 'Folk', 2: 'Indie', 3: 'Islamic', 4: 'Hiphop', 5: 'Rock', 6: 'Metal', 7: 'Pop'} 

 {'Adhunik': 0, 'Folk': 1, 'Indie': 2, 'Islamic': 3, 'Hiphop': 4, 'Rock': 5, 'Metal': 6, 'Pop': 7}


# Load and preprocess data

In [ ]:
from pathlib import Path
import torchaudio

# def load_data():
#     file_list = []
#     label_list = []

#     for folder in Path('/kaggle/input/banglabeats3sec/wavs3sec/').glob('*'):
#         genre = folder.name  # Get the genre name from the folder

#         # Count the number of files processed for this genre
#         files_processed = 0

#         for file in folder.glob('*.wav'):
#             if not 'jazz.00054' in str(file):  # skip bad example
#                 file_list.append(file)
#                 label_list.append(label2id[genre])
#                 files_processed += 1

#                 if files_processed >= 1700:
#                     break  # Break out of the loop if the limit is reached

#         if files_processed >= limit_per_folder:
#             break  # Break out of the outer loop if the limit is reached for all genres

#     dd = pd.DataFrame()
#     dd['file'] = file_list
#     dd['label'] = label_list
#     return dd

from pathlib import Path
import torchaudio

def load_data():
    file_list = []
    label_list = []
    for file in Path('/kaggle/input/banglabeats3sec/wavs3sec/').glob('*/*.wav'):
        if not 'jazz.00054' in str(file): # skip bad example
            genre = file.parent.name
            file_list.append(file)
            label_list.append(label2id[genre])
    dd = pd.DataFrame()
    dd['file'] = file_list
    dd['label'] = label_list
    return dd

In [ ]:
%%time
dd = load_data()
dd.head()

CPU times: user 291 ms, sys: 33.6 ms, total: 324 ms
Wall time: 1.27 s


,file,label
0,/kaggle/input/banglabeats3sec/wavs3sec/Metal/6...,6
1,/kaggle/input/banglabeats3sec/wavs3sec/Metal/1...,6
2,/kaggle/input/banglabeats3sec/wavs3sec/Metal/3...,6
3,/kaggle/input/banglabeats3sec/wavs3sec/Metal/3...,6
4,/kaggle/input/banglabeats3sec/wavs3sec/Metal/1...,6


In [ ]:
dd.shape

(16170, 2)

In [ ]:
# # random oversampling of minority class # ensure that each class has the same number of records
# y = dd[['label']]
# dd = dd.drop(['label'], axis=1)
# ros = RandomOverSampler(random_state=83)
# dd, y_resampled = ros.fit_resample(dd, y)
# del y
# dd['label'] = y_resampled
# del y_resampled
# gc.collect()

In [ ]:
dd.shape, dd['label'].value_counts()

((16170, 2),
 1    2350
 0    2230
 3    2050
 2    2020
 7    1960
 6    1920
 5    1910
 4    1730
 Name: label, dtype: int64)

In [ ]:
gc.collect()
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [ ]:
def get_transform_audio(file):
    audio,rate = torchaudio.load(str(file))
    transform = torchaudio.transforms.Resample(rate,RATE_HZ)
    audio = transform(audio).squeeze(0).numpy()
    audio = audio[:MAX_LENGTH]
    return audio # truncate to first part of audio to save RAM
dd['audio'] = dd['file'].progress_apply(get_transform_audio)

100%|██████████| 16170/16170 [08:26<00:00, 31.90it/s]


In [ ]:
%%time
dd = dd.drop(['file'], axis=1)

CPU times: user 292 µs, sys: 1.95 ms, total: 2.25 ms
Wall time: 4.58 ms


In [ ]:
dd.sample(5)

,label,audio
7664,0,"[-0.10168226, -0.19984412, -0.14819269, -0.097..."
10072,2,"[-0.21388151, -0.34130138, -0.26057282, -0.206..."
14896,7,"[-0.0077559645, -0.013353276, -0.019928586, -0..."
14794,7,"[-0.14665158, -0.15244454, -0.08240593, -0.172..."
14158,4,"[0.008032747, 0.07527016, 0.18997057, 0.186019..."


In [ ]:
gc.collect()
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [ ]:
from datasets import Dataset, ClassLabel
dd = Dataset.from_pandas(dd)

In [ ]:
from collections import Counter
Counter(dd['label']).items()

dict_items([(6, 1920), (1, 2350), (5, 1910), (0, 2230), (2, 2020), (3, 2050), (4, 1730), (7, 1960)])

In [ ]:
dd = dd.train_test_split(test_size=0.2)
dd

DatasetDict({
    train: Dataset({
        features: ['label', 'audio'],
        num_rows: 12936
    })
    test: Dataset({
        features: ['label', 'audio'],
        num_rows: 3234
    })
})

In [ ]:
gc.collect()

31

# Load facebook/wav2vec2-base-960h model

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

model_str = "facebook/wav2vec2-base-960h"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_str)
model = AutoModelForAudioClassification.from_pretrained(model_str,num_labels=len(labels))
model.config.id2label = id2label
# number of trainable parameters
print(model.num_parameters(only_trainable=True)/1e6)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForSequenceClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['projector.bias', 'wav2vec2.masked_spec_embed', 'classifier.weight', 'classifier.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be 

94.570632


In [ ]:
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [ ]:
gc.collect()

420

In [ ]:
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
gc.collect()

0

In [ ]:
def preprocess_function(batch):
    inputs = feature_extractor(batch['audio'], sampling_rate=RATE_HZ, max_length=MAX_LENGTH, truncation=True)
    inputs['input_values'] = inputs['input_values'][0]
    return inputs

dd['test'] = dd['test'].map(preprocess_function, remove_columns="audio", batched=False)
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
gc.collect()
dd['train'] = dd['train'].map(preprocess_function, remove_columns="audio", batched=False)


Map:   0%|          | 0/3234 [00:00<?, ? examples/s]

Map:   0%|          | 0/12936 [00:00<?, ? examples/s]

In [ ]:
gc.collect()
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

# Train and evaluate model

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

from sklearn.metrics import roc_auc_score
def compute_metrics(eval_pred):
    # Compute the ROC AUC score
    predictions = eval_pred.predictions
    predictions = np.exp(predictions)/np.exp(predictions).sum(axis=1, keepdims=True)
    label_ids = eval_pred.label_ids
    roc_auc = roc_auc_score(label_ids, predictions, average='macro', multi_class='ovr') # one-vs-rest ROC AUC score

    # Calculate accuracy using the loaded accuracy metric
    acc_score = accuracy.compute(predictions=predictions.argmax(axis=1), references=label_ids)['accuracy']

    return {
        "roc_auc": roc_auc,
        "accuracy": acc_score
    }

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer
batch_size=4
warmup_steps=50
weight_decay=0.02
num_train_epochs=25
model_name = "wav2vec2-base-960h"
training_args = TrainingArguments(
    f"{model_name}-finetuned-banglabeats",
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-5, # 5e-6
    logging_strategy='steps',
    logging_first_step=True,
    load_best_model_at_end=True,
    logging_steps=1,
    evaluation_strategy='epoch',
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    eval_steps=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    save_strategy='epoch',
    push_to_hub=True,
    save_total_limit=1, # save fewer checkpoints to limit used space
    report_to="mlflow",  # log to mlflow
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dd["train"],
    eval_dataset=dd["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/thisisjibon/wav2vec2-base-960h-finetuned-banglabeats into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.40k/361M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.55k/3.55k [00:00<?, ?B/s]

Clean file training_args.bin:  28%|##8       | 1.00k/3.55k [00:00<?, ?B/s]


KeyboardInterrupt



In [ ]:
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
gc.collect()

In [ ]:
trainer.evaluate()

In [ ]:
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
gc.collect()

In [ ]:
trainer.train()

In [ ]:
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
gc.collect()

In [ ]:
trainer.evaluate()

In [ ]:
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
gc.collect()

In [ ]:
trainer.save_model()

In [ ]:
model_name = "wav2vec2-base-960h"
kwargs = {
    "dataset_tags": "thisisjibon/banglabeats",
    "dataset": "BanglaBeats",
    "model_name": f"{model_name}-finetuned-banglabeats",
    "finetuned_from": "facebook/wav2vec2-base-960h",
    "tasks": "audio-classification",
}
trainer.push_to_hub(**kwargs)

In [ ]:
# from transformers import pipeline

# pipe=pipeline('audio-classification',model=model_name,device=0)

In [ ]:
# # disco example
# audio,rate=torchaudio.load('/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00005.wav')
# transform=torchaudio.transforms.Resample(rate,RATE_HZ)
# audio=transform(audio).numpy().reshape(-1)
# # make a classification pipeline
# pipe(audio)

In [ ]:
# # classical example
# audio,rate=torchaudio.load('/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/classical/classical.00014.wav')
# transform=torchaudio.transforms.Resample(rate,RATE_HZ)
# audio=transform(audio).numpy().reshape(-1)
# # make a classification pipeline
# pipe(audio)

In [ ]:
# from IPython.display import Audio
# Audio(audio,rate=RATE_HZ)

# Send model to Huggingface

In [ ]:
# # finally, save the model to Huggingface
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
# from huggingface_hub import HfApi
# api = HfApi()
# repo_id = f"thisisjibon/{model_name}"
# try:
#     api.create_repo(repo_id)
# except:
#     print(f"Repo {repo_id} already exists")

In [ ]:
# api.upload_folder(
#     folder_path=model_name,
#     path_in_repo = ".",
#     repo_id=repo_id,
#     repo_type="model"
# )